In [1]:
import sqlite3
import pandas as pd

In [2]:
def get_drivers_with_constructor_relationship(conn: str) -> pd.DataFrame:
    """
    Get drivers per year with their constructor for that year.

    :param conn: The database connection.
    :return: DataFrame with year, driverId, constructorId, and constructorName.
    """
    query = """
        SELECT races.year, r.driverId, r.constructorId, c.name AS constructorName, c.nationality AS constructor_nationality
        FROM results r
        JOIN races ON r.raceId = races.raceId
        JOIN constructors c ON r.constructorId = c.constructorId
        WHERE races.year <> 2023
        GROUP BY r.driverId, races.year, r.constructorId, constructorName;
    """
    df = pd.read_sql_query(query, conn)

    return df


In [3]:
def get_constructor_points(conn: str) -> pd.DataFrame:
    """
    Calculate constructor points, including rank and number of times in the first place.

    :param conn: The database connection.
    :return: DataFrame year, driverId and constructorId.
    - DataFrame containing constructor points with additional information.
    """
    constructor_points_query = '''
        WITH LastRacePerSeason AS (
            SELECT
                r.year,
                MAX(r.date) AS last_race_date
            FROM
                races r
            WHERE
                r.year BETWEEN 1950 AND 2022
            GROUP BY
                r.year
        ),
        ConstructorStandingsWithRank AS (
            SELECT
                lrs.year,
                cs.constructorId,
                cs.points AS constructor_points,
                RANK() OVER (PARTITION BY lrs.year ORDER BY cs.points DESC) AS constructor_place
            FROM
                LastRacePerSeason lrs
            JOIN
                races r ON lrs.year = r.year AND lrs.last_race_date = r.date
            JOIN
                constructor_standings cs ON r.raceId = cs.raceId
            JOIN
                constructors c ON cs.constructorId = c.constructorId
        )

        SELECT
            *
        FROM
            ConstructorStandingsWithRank
        ORDER BY
            year DESC, constructor_points DESC;
    '''

    df = pd.read_sql_query(constructor_points_query, conn)

    return df